#### score2 : 
**log(1+ 전체 확진자수/200) * (해당 시의 확진자 수/전체 확진자 수) * (1.005-log(해당 사건 지속 기간)) / 사건 해당 횟수**

대구 신천지 02.19

청도 대남병원(경북) 02.20

분당 재생병원 03.06

구로 콜센터 03.10

이태원 05.07

삼성서울병원 05.19

부천 쿠팡 05.26

In [2]:
import numpy as np
import pandas as pd

covid = pd.read_excel('all_covid.xlsx')
seoul = pd.read_excel('seoul_covid.xlsx')
daegu = pd.read_excel('daegu_covid.xlsx')

# 대구 빈 값 0으로 채우기
daegu = daegu.fillna(0)
# DATE를 string형태로 바꾸기
covid['DATE'] = covid['DATE'].astype(str)
seoul['DATE'] = seoul['DATE'].astype(str)
daegu['DATE'] = daegu['DATE'].astype(str)

In [5]:
# (1.005-log(해당 사건 지속 기간)) / 사건 해당 횟수

idx, i = 0, 1
case_date = ['2020-02-19','2020-02-20','2020-03-06','2020-03-10','2020-05-07','2020-05-19','2020-05-26']
flag = True
case_cnt, case_time = [], []

for date in covid['DATE']:
    try: # index 에러는 무시해도 됨
        if date == case_date[idx]:
            flag = False
            i = 1 
            idx +=1
            case_time.append((1.005 - np.log(i)) / idx)
            case_cnt.append(idx)

        else:
            if flag:
                case_time.append((1.005 - np.log(0)) / 0.0000001)
                case_cnt.append(0)
            else:
                i+=1
                case_time.append((1.005 - np.log(i)) / idx)
                case_cnt.append(idx)
    except:
        i+=1
        case_time.append((1.005 - np.log(i)) / idx)
        case_cnt.append(idx)

# print(len(case_time))
# print(len(case_cnt))

c:\users\lg\.conda\envs\bok\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in log


In [6]:
covid_seoul = pd.merge(covid, seoul, on='DATE')
covid_seoul['case_cnt'] = case_cnt
covid_seoul['case_time'] = case_time
# 2월 1일부터 2월 18일까지는 0으로 처리
covid_seoul['case_time'].replace(np.inf, 0, inplace=True)


covid_daegu = pd.merge(covid, daegu, on='DATE')
covid_daegu['case_cnt'] = case_cnt
covid_daegu['case_time'] = case_time
# 2월 1일부터 2월 18일까지는 0으로 처리
covid_seoul['case_time'].replace(np.inf, 0, inplace=True)

In [12]:
# log(1+ 전체 확진자수/200)
def first_sik(n):
    a = np.log(1+ n/200)
    return a
    
# (해당 시의 확진자 수/전체 확진자 수)
def second_sik(n, si_n):
    if n == 0:
        return 0
    else:
        b = si_n/n
        return b

# log(1+ 전체 확진자수/200) * (해당 시의 확진자 수/전체 확진자 수) * (1.005-log(해당 사건 지속 기간)) / 사건 해당 횟수
def main_func(dataframe, si_name):

    score = []

    for data in (covid_seoul.iterrows()):

        a = first_sik(data[1][1])
        b = second_sik(data[1][1], data[1][2])
        mul = (a * b * data[1][4])
        score.append(mul)
        
    df = pd.DataFrame({'DATE':dataframe['DATE'],
                       'SCORE' : score})
    df.to_csv(si_name+'.csv', index=False)
        
main_func(covid_seoul, 'seoul_score')
main_func(covid_daegu, 'daegu_score')

In [13]:
seoul_score = pd.read_csv('seoul_score.csv')
seoul_score

,DATE,SCORE
0,2020-02-01,0.000000
1,2020-02-02,0.000000
2,2020-02-03,0.000000
3,2020-02-04,0.000000
4,2020-02-05,0.000000
...,...,...
116,2020-05-27,0.004505
117,2020-05-28,-0.001174
118,2020-05-29,-0.002239
119,2020-05-30,-0.002430


In [14]:
daegu_score = pd.read_csv('daegu_score.csv')
daegu_score

,DATE,SCORE
0,2020-02-01,0.000000
1,2020-02-02,0.000000
2,2020-02-03,0.000000
3,2020-02-04,0.000000
4,2020-02-05,0.000000
...,...,...
116,2020-05-27,0.004505
117,2020-05-28,-0.001174
118,2020-05-29,-0.002239
119,2020-05-30,-0.002430
